# Day 9: Advanced SQL Analytics & Business Queries

## Setup - Tables and Dummy Data

In [1]:

import sqlite3
import pandas as pd

conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# Create tables
cursor.execute('''CREATE TABLE customers (
    customer_id INTEGER PRIMARY KEY,
    name TEXT, country TEXT
)''')

cursor.execute('''CREATE TABLE products (
    product_id INTEGER PRIMARY KEY,
    product_name TEXT, category TEXT, price REAL
)''')

cursor.execute('''CREATE TABLE orders (
    order_id INTEGER PRIMARY KEY,
    customer_id INTEGER,
    product_id INTEGER,
    amount REAL,
    order_date TEXT,
    FOREIGN KEY(customer_id) REFERENCES customers(customer_id),
    FOREIGN KEY(product_id) REFERENCES products(product_id)
)''')

# Insert data
cursor.executescript('''
INSERT INTO customers VALUES (1, 'Alice', 'India');
INSERT INTO customers VALUES (2, 'Bob', 'USA');
INSERT INTO customers VALUES (3, 'Charlie', 'Canada');

INSERT INTO products VALUES (1, 'Laptop', 'Electronics', 70000);
INSERT INTO products VALUES (2, 'Tablet', 'Electronics', 30000);
INSERT INTO products VALUES (3, 'Chair', 'Furniture', 5000);
INSERT INTO products VALUES (4, 'Desk', 'Furniture', 8000);

INSERT INTO orders VALUES (101, 1, 1, 72000, '2024-06-10');
INSERT INTO orders VALUES (102, 2, 2, 31000, '2024-06-15');
INSERT INTO orders VALUES (103, 1, 2, 30000, '2024-07-10');
INSERT INTO orders VALUES (104, 3, 3, 4800, '2024-07-18');
INSERT INTO orders VALUES (105, 2, 4, 8200, '2024-08-01');
INSERT INTO orders VALUES (106, 1, 1, 70000, '2024-08-05');
INSERT INTO orders VALUES (107, 3, 1, 70000, '2024-08-09');
''')
conn.commit()


## Multi-table Join Analysis

In [2]:

query1 = '''
SELECT c.name, c.country, COUNT(o.order_id) AS total_orders, SUM(o.amount) AS total_spent
FROM customers c
JOIN orders o ON c.customer_id = o.customer_id
GROUP BY c.customer_id
'''
pd.read_sql_query(query1, conn)


,name,country,total_orders,total_spent
0,Alice,India,3,172000.0
1,Bob,USA,2,39200.0
2,Charlie,Canada,2,74800.0


## Window Functions

In [3]:

query2 = '''
SELECT order_id, customer_id, amount,
RANK() OVER(PARTITION BY customer_id ORDER BY amount DESC) AS rank_by_amount
FROM orders
'''
pd.read_sql_query(query2, conn)


,order_id,customer_id,amount,rank_by_amount
0,101,1,72000.0,1
1,106,1,70000.0,2
2,103,1,30000.0,3
3,102,2,31000.0,1
4,105,2,8200.0,2
5,107,3,70000.0,1
6,104,3,4800.0,2


In [4]:

query3 = '''
SELECT strftime('%Y-%m', order_date) as month, amount,
SUM(amount) OVER (ORDER BY order_date ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS running_total
FROM orders
'''
pd.read_sql_query(query3, conn)


,month,amount,running_total
0,2024-06,72000.0,72000.0
1,2024-06,31000.0,103000.0
2,2024-07,30000.0,133000.0
3,2024-07,4800.0,137800.0
4,2024-08,8200.0,146000.0
5,2024-08,70000.0,216000.0
6,2024-08,70000.0,286000.0


## CTEs

In [5]:

query4 = '''
WITH avg_orders AS (
    SELECT customer_id, AVG(amount) AS avg_amt
    FROM orders
    GROUP BY customer_id
)
SELECT c.name, a.avg_amt
FROM avg_orders a
JOIN customers c ON a.customer_id = c.customer_id
WHERE avg_amt > 10000
'''
pd.read_sql_query(query4, conn)


,name,avg_amt
0,Alice,57333.333333
1,Bob,19600.000000
2,Charlie,37400.000000


In [6]:

query5 = '''
WITH recent_orders AS (
    SELECT * FROM orders
    WHERE order_date >= date('2024-06-01')
)
SELECT strftime('%Y-%m', order_date) AS month, SUM(amount) AS revenue
FROM recent_orders
GROUP BY month
'''
pd.read_sql_query(query5, conn)


,month,revenue
0,2024-06,103000.0
1,2024-07,34800.0
2,2024-08,148200.0


## Business Scenario - Monthly Revenue by Product

In [7]:

query6 = '''
SELECT p.product_name, strftime('%Y-%m', o.order_date) AS month, SUM(o.amount) AS monthly_sales
FROM orders o
JOIN products p ON o.product_id = p.product_id
GROUP BY p.product_name, month
ORDER BY p.product_name, month
'''
pd.read_sql_query(query6, conn)


,product_name,month,monthly_sales
0,Chair,2024-07,4800.0
1,Desk,2024-08,8200.0
2,Laptop,2024-06,72000.0
3,Laptop,2024-08,140000.0
4,Tablet,2024-06,31000.0
5,Tablet,2024-07,30000.0
